In [1]:
import json
import requests
import csv
import jsonlines
import sys
import numpy as np
import random
import math
import openai
import re
import pandas as pd
import os
from numpy.linalg import norm
import itertools
import string
import torch
from transformers import BertTokenizer, BertModel
import logging
import gensim.downloader as api
import sys
sys.path.insert(0, './Metrics')
from Automatic_Metrics import Scorers
from Linguistic_Metrics import LinguisticFeatures
import Stylistic_Metrics as st
import  Metrics.Stylistic_Metrics
import nltk
import replicate

nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\aryam\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [3]:
clean_data = json.load(open(r"Data\data_preprocessed.json"))
data_reduced = clean_data[0:50]

In [4]:
os.environ['REPLICATE_API_TOKEN'] = 'r8_doCPWxyRN9K3zjdjl4PhZXq62gwmNrM0xBcAG'

In [5]:
def call_replicate(prompt):
    output = replicate.run(
        "meta/llama-2-7b-chat:f1d50bb24186c52daae319ca8366e53debdaa9e0ae7ff976e918df752732ccc4",
        input={
            "top_p": 1,
            "prompt": prompt,
            "temperature": 0.75
            # "repetition_penalty": 1,
            # "stop_sequences": stop_sequences
        }
    )
    # Collecting and processing the output
    output_text = "".join(str(item) for item in output)
    return output_text.strip()

Prompt 1

In [6]:
def Prompt_1(contents,question):
    messages = f"""
                [INST] <<SYS>>You are a NLP domain expert question answering system.<</SYS>>\n
                Contents of a research paper: {contents}.
                Question to be answered: {question}. 
                The answer should strictly be an exact extracted text from the contents.
                If an exact extracted text does not answer the question, answer with a brief and precise answer derived from the contents.
                Strictly answer in one sentence or less.
                Reply in the format:
                "answer" [\INST]
            """  
    events = call_replicate(messages)
    return events

Prompt 2

In [7]:
def Prompt_2(contents,question):
    messages = f"""
                [INST] <<SYS>>You are a question-answering system<</SYS>>\n
                Contents of a research paper: {contents}.
                Question to be answered: {question}.
                Strictly answer in one sentence or less. 
                The answer should strictly be an exact extracted text from the contents.
                If an exact extracted text does not answer the question, answer with a brief and precise answer derived from the contents.
                Reply in the format:
                "answer" [\INST]
            """  
    events = call_replicate(messages)
    return events

Prompt 3

In [8]:
def Prompt_3(contents,question):
    messages = f"""
                [INST] Answer the question provided based on the contents in 1 sentence.
                Contents of a research paper: {contents}.
                Question to be answered: {question}.
                Strictly answer in one sentence or less.
                [\INST]
            """  
    events = call_replicate(messages)
    return events

Prompt 4

In [9]:
def Prompt_4(contents,question):
    messages = f"""
                [INST] <<SYS>>You are a child who has to answer a question with no knowledge of the domain.<</SYS>>\n
                Contents of a research paper: {contents}.
                Question to be answered: {question}.
                Strictly answer the question in one sentence or less. 
                The answer should strictly be an exact extracted text from the contents.
                If an exact extracted text does not answer the question, answer like a child reading the contents.
                Reply in the format:
                "answer" [\INST]
            """  
    events = call_replicate(messages)
    return events

In [12]:
Prompt_4("To enrich publicly available non-collaborative task datasets, we collect a new dataset AntiScam, where users defend themselves against attackers trying to collect personal information. As non-collaborative tasks are still relatively new to the study of dialog systems, there are insufficiently many meaningful datasets for evaluation and we hope this provides a valuable example. We evaluate MISSA on the newly collected AntiScam dataset and an existing PersuasionForGood dataset. Both automatic and human evaluations suggest that MISSA outperforms multiple competitive baselines. To enrich available non-collaborative task datasets, we created a corpus of human-human anti-scam dialogs in order to learn human elicitation strategies. We chose a popular Amazon customer service scam scenario to collect dialogs between users and attackers who aim to collect users information. We posted a role-playing task on the Amazon Mechanical Turk platform and collected a typing conversation dataset named AntiScam. We collected 220 human-human dialogs. The average conversation length is 12.45 turns and the average utterance length is 11.13 words. Only 172 out of 220 users successfully identified their partner as an attacker, suggesting that the attackers are well trained and not too easily identifiable. We recruited two expert annotators who have linguistic training to annotate 3,044 sentences in 100 dialogs, achieving a 0.874 averaged weighted kappa value. Datasets ::: PersuasionForGood Dataset The PersuasionForGood dataset BIBREF1 was collected from typing conversations on Amazon Mechanical Turk platform. Two workers were randomly paired, one was assigned the role of persuader, the other was persuadee. The goal of the persuader was to persuade the persuadee to donate a portion of task earning to a specific charity. The dataset consists of 1,017 dialogs, where 300 dialogs are annotated with dialog acts. The average conversation length is 10.43, the vocabulary size is 8,141. Since the original PersuasionForGood dataset is annotated with dialog acts, we select the on-task dialog acts as on-task intents shown in Table TABREF2, and categorize the other dialog acts into our pre-defined off-task intents. FLOAT SELECTED: Table 1: Hierarchical intent annotation scheme on both ANTISCAM dataset and PERSUASIONFORGOOD dataset. The On-task intents are task-specific while the Off-task intents are general for different non-collaborative tasks. To decouple syntactic and semantic information in utterances and provide detailed supervision, we design a hierarchical intent annotation scheme for non-collaborative tasks. We first separate on-task and off-task intents. As on-task intents are key actions that can vary among different tasks, we need to specifically define on-task intents for each task. On the other hand, since off-task content is too general to design task-specific intents, we choose common dialog acts as the categories. The advantage of this hierarchical annotation scheme is apparent when starting a new non-collaborative task: we only need to focus on designing the on-task categories and semantic slots which are the same as traditional task-oriented dialog systems. Consequently, we don't have to worry about the off-task annotation design since the off-task category is universal. In the intent annotation scheme shown in Table TABREF2, we list the designed intent annotation scheme for the newly collected AntiScam dataset and the PersuasionForGood dataset. We first define on-task intents for the datasets, which are key actions in the task. Since our AntiScam focuses on understanding and reacting towards elicitations, we define elicitation, providing_information and refusal as on-task intents. In the PersuasionForGood dataset, we define nine on-task intents in Table TABREF2 based on the original PersuasionForGood dialog act annotation scheme. All these intents are related to donation actions, which are salient on-task intents in the persuasion task. The off-task intents are the same for both tasks, including six general intents and six additional social intents. General intents are more closely related to the syntactic meaning of the sentence (open_question, yes_no_question, positive_answer, negative_answer, responsive_statement, and nonresponsive_statement) while social intents are common social actions (greeting, closing, apology, thanking,respond_to_thank, and hold). For specific tasks, we also design a semantic slot annotation scheme for annotating sentences based on their semantic content. We identify 13 main semantic slots in the anti-scam task, for example, credit card numbers. We present a detailed semantic slot annotation in Table TABREF3. Following BIBREF1, we segment each conversation turn into single sentences and then annotate each sentence rather than turns. To tackle the issue of incoherent system responses to off-task content, previous studies have built hybrid systems to interleave off-task and on-task content. BIBREF4 used a rule-based dialog manager for on-task content and a neural model for off-task content, and trained a reinforcement learning model to select between these two models based on the dialog context. However, such a method is difficult to train and struggles to generalize beyond the movie promotion task they considered. To tackle these problems, we propose a hierarchical intent annotation scheme that separates on-task and off-task information in order to provide detailed supervision. For on-task information, we directly use task-related intents for representation. Off-task information, on the other hand, is too general to categorize into specific intents, so we choose dialog acts that convey syntax information. These acts, such as \u201copen question\" are general to all tasks.","How is intent annotated?")

'"The intent is annotated using a hierarchical intent annotation scheme that separates on-task and off-task information. On-task intents are key actions that can vary among different tasks, while off-task content is too general to design task-specific intents. The on-task intents are specifically defined for each task, while the off-task content is categorized into general dialog acts."'

In [10]:
for i,datum in enumerate(data_reduced):
    print('{}/{}'.format(i, len(data_reduced)))
    # example_content = "To enrich available non-collaborative task datasets, we created a corpus of human-human anti-scam dialogs in order to learn human elicitation strategies. We chose a popular Amazon customer service scam scenario to collect dialogs between users and attackers who aim to collect users information. We posted a role-playing task on the Amazon Mechanical Turk platform and collected a typing conversation dataset named AntiScam. We collected 220 human-human dialogs. The average conversation length is 12.45 turns and the average utterance length is 11.13 words. Only 172 out of 220 users successfully identified their partner as an attacker, suggesting that the attackers are well trained and not too easily identifiable. We recruited two expert annotators who have linguistic training to annotate 3,044 sentences in 100 dialogs, achieving a 0.874 averaged weighted kappa value."
    for qa in datum['qas']:
        contents = qa['evidence']
        LLM_answer = Prompt_4(contents,qa['question'])
        print(LLM_answer)
        qa['GPT_Answer'] = LLM_answer
save_json(data_reduced, r'Data\LLama_P4.json')

0/50


The ANTISCAM dataset contains 220 human-human dialogs with an average conversation length of 12.45 turns and an average utterance length of 11.13 words.
According to the provided text, intent is annotated using a hierarchical intent annotation scheme. This scheme separates on-task and off-task intents, with on-task intents being specifically defined for each task and off-task intents being general and universal. On-task intents are key actions that can vary among different tasks, while off-task content is too general to be specifically categorized. The annotation scheme includes on-task intents such as elicitation, providing_information, and refusal, as well as off-task intents including general intents and social intents.
According to the contents of the research paper, the baseline models outperformed by the proposed work, MISSA, are TransferTransfo and a hybrid model that combines vanilla TransferTransfo and MISSA.
According to the contents, the evaluation metrics and criteria used 

In [11]:
data_final = json.load(open(r"Data\LLama_P4.json"))

In [12]:
scorer = Scorers()
rel = st.relevance()
fl = st.fluency()
cr = st.Correctness()
for i,datum in enumerate(data_final):
    print('{}/{}'.format(i, len(data_final)))
    for qa in datum['qas']:
        bert_scores = []
        ner_overlap = []
        writer_summ = qa['answers']
        llm_summ = qa['GPT_Answer']
        readability = st.Readability(llm_summ)
        formality = st.Formality(llm_summ)
        bleu_1,bleu_2,bleu_3,bleu_4 = scorer.compute_bleu(writer_summ,llm_summ)
        
        qa['bleu_1'] = bleu_1
        qa['bleu_2'] = bleu_2
        qa['bleu_3'] = bleu_3
        qa['bleu_4'] = bleu_4

        rogue = scorer.compute_rouge(writer_summ,llm_summ)
        qa['rogue'] = rogue

        meteor = scorer.compute_meteor(writer_summ,llm_summ)
        qa['meteor'] = meteor
        qa['QA_Relevance'] = rel.default(qa['question'],qa['GPT_Answer'])
        qa['CA_Relevance'] = rel.default(qa['evidence'],qa['GPT_Answer'])
        qa['fluency'] = fl.default(qa['GPT_Answer'])
        for ref in writer_summ:
        #     # bert_scores.append(scorer.Bert_Score([ref],[llm_summ]))
            ner_overlap.append(cr.default(ref,qa['GPT_Answer']))
        qa['Correctness'] = max(ner_overlap)
        # qa['bert_score'] = max(bert_scores)
        qa['readability_LLM'] = readability.default()
        qa['formality_LLM'] = formality.default()

save_json(data_final, r'Data/LLama_P4.json')

c:\Users\aryam\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


0/50


c:\Users\aryam\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1/50


c:\Users\aryam\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\aryam\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


2/50


c:\Users\aryam\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


3/50
4/50
5/50
6/50
7/50
8/50
9/50
10/50
11/50
12/50
13/50
14/50
15/50
16/50
17/50
18/50
19/50
20/50
21/50
22/50
23/50
24/50
25/50
26/50
27/50
28/50
29/50
30/50
31/50
32/50
33/50
34/50
35/50
36/50
37/50
38/50
39/50
40/50
41/50
42/50
43/50
44/50
45/50
46/50
47/50
48/50
49/50


In [13]:
temp = json.load(open(r"Data\LLama_P4.json"))
max_rogue_1_f = 0.0
max_rogue_2_f = 0.0
max_rogue_l_f = 0.0
for i,datum in enumerate(temp):
    print('{}/{}'.format(i, len(temp)))
    for qa in datum['qas']:
        for rogue_list in qa["rogue"]:
            for rogue_dict in rogue_list:
                max_rogue_1_f = max(max_rogue_1_f, rogue_dict["rouge-1"]["f"])
                max_rogue_2_f = max(max_rogue_2_f, rogue_dict["rouge-2"]["f"])
                max_rogue_l_f = max(max_rogue_l_f, rogue_dict["rouge-l"]["f"])
        qa['rogue'] = {"rogue-1":max_rogue_1_f,"rogue-2":max_rogue_2_f,"rogue-l":max_rogue_l_f}
save_json(temp, r'Data/results/LLama_P4.json')

0/50
1/50
2/50
3/50
4/50
5/50
6/50
7/50
8/50
9/50
10/50
11/50
12/50
13/50
14/50
15/50
16/50
17/50
18/50
19/50
20/50
21/50
22/50
23/50
24/50
25/50
26/50
27/50
28/50
29/50
30/50
31/50
32/50
33/50
34/50
35/50
36/50
37/50
38/50
39/50
40/50
41/50
42/50
43/50
44/50
45/50
46/50
47/50
48/50
49/50


In [20]:
data = json.load(open(r"Data\responses_P4.json"))
references=[]
candidates = []
for i,datum in enumerate(data):
    print('{}/{}'.format(i, len(data_final)))

    for qa in datum['qas']:
        references.append(qa["answers"])
        candidates.append([qa["GPT_Answer"]])
print(references)
print(candidates)

0/50
1/50
2/50
3/50
4/50
5/50
6/50
7/50
8/50
9/50
10/50
11/50
12/50
13/50
14/50
15/50
16/50
17/50
18/50
19/50
20/50
21/50
22/50
23/50
24/50
25/50
26/50
27/50
28/50
29/50
30/50
31/50
32/50
33/50
34/50
35/50
36/50
37/50
38/50
39/50
40/50
41/50
42/50
43/50
44/50
45/50
46/50
47/50
48/50
49/50
[[' 3,044 sentences in 100 dialogs', '220 human-human dialogs', '220 human-human dialogs. ', '3,044 sentences in 100 dialogs', '  220 human-human dialogs.  3,044 sentences in 100 dialogs', '220 human-human dialogs. The average conversation length is 12.45 turns and the average utterance length is 11.13 words. ', '3,044 sentences in 100 dialogs'], ['using a role-playing task on the Amazon Mechanical Turk platform and collecting typed conversations', 'Separate on-task and off task intents and annotate on task for data set specific intents, while annotating  off task intents with a fixed set of general intents.', 'On-task dialog are annotated as on-task intents , the other dialog are annotated as pre-def

In [61]:
import nltk
BLEUscore = nltk.translate.bleu_score.corpus_bleu(references, candidates)
BLEUscore

0